In [2]:
import pandas as pd

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [7]:
data= pd.read_csv('train.csv')

In [8]:
data.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [9]:
features = data.drop(["Cover_Type", "Id"], axis=1)
target = data["Cover_Type"]

In [12]:
model = RandomForestClassifier(n_estimators=10)

In [13]:
kfold = KFold(n_splits=10, random_state=0, shuffle=True)

In [14]:
cv_result = cross_val_score(
        model, features, target, cv=kfold, scoring="accuracy"
    )

In [15]:
cv_result

array([0.83134921, 0.83399471, 0.83730159, 0.82275132, 0.82473545,
       0.85648148, 0.83994709, 0.84656085, 0.83796296, 0.84722222])

In [24]:
def create_pipeline(
    use_scaler: bool, random_state: int
) -> Pipeline:
    pipeline_steps = []
    if use_scaler:
        pipeline_steps.append(("scaler", StandardScaler()))
    pipeline_steps.append(
        (
            "classifier",
            RandomForestClassifier(
                random_state=random_state
            ),
        )
    )
    return Pipeline(steps=pipeline_steps)

In [25]:
pipeline = create_pipeline(True, 42)
pipeline.fit(features, target)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', RandomForestClassifier(random_state=42))])

In [28]:
data_test= pd.read_csv('test.csv').set_index('Id')
data_test.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
Id,,,,,,,,,,,,,,,,,,,,,
15121,2680,354,14,0,0,2684,196,214,156,6645,...,0,0,0,0,0,0,0,0,0,0
15122,2683,0,13,0,0,2654,201,216,152,6675,...,0,0,0,0,0,0,0,0,0,0
15123,2713,16,15,0,0,2980,206,208,137,6344,...,0,0,0,0,0,0,0,0,0,0
15124,2709,24,17,0,0,2950,208,201,125,6374,...,0,0,0,0,0,0,0,0,0,0
15125,2706,29,19,0,0,2920,210,195,115,6404,...,0,0,0,0,0,0,0,0,0,0


In [30]:
y_pred = pipeline.predict(data_test)

In [33]:
df = pd.DataFrame({'Id':data_test.index, 'Cover_Type':y_pred})
df.set_index('Id', inplace=True)
df.to_csv(r'sampleSubmission.csv')